<a href="https://colab.research.google.com/github/VadrevuSaiSanjana/smart-issue-board/blob/main/smart_issue_board_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Intelligent Support Ticket Manager
# Built using Python + SQL + Text Similarity
# Author: Sai Sanjana Vadrevu

import sqlite3
from datetime import datetime
from math import sqrt

# -------------------------------
# DATABASE INITIALIZATION
# -------------------------------
db = sqlite3.connect("support_tickets.db")
cur = db.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS tickets (
    ticket_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    description TEXT,
    priority TEXT,
    status TEXT,
    assigned_to TEXT,
    created_by TEXT,
    created_time TEXT
)
""")
db.commit()

# -------------------------------
# TEXT SIMILARITY (CUSTOM LOGIC)
# -------------------------------
def text_similarity(text1, text2):
    words1 = set(text1.lower().split())
    words2 = set(text2.lower().split())

    common = words1.intersection(words2)
    total = len(words1) + len(words2)

    if total == 0:
        return 0

    return (2 * len(common)) / total   # Dice coefficient

# -------------------------------
# CHECK DUPLICATE ISSUES
# -------------------------------
def is_similar_issue(new_text):
    cur.execute("SELECT title, description FROM tickets")
    existing = cur.fetchall()

    for t, d in existing:
        similarity_score = text_similarity(new_text, t + " " + d)
        if similarity_score > 0.4:
            return True
    return False

# -------------------------------
# CREATE TICKET
# -------------------------------
def create_ticket(title, description, priority, assigned_to, created_by):
    combined = title + " " + description

    if is_similar_issue(combined):
        print("⚠️ Similar issue detected. Please verify before proceeding.")

    cur.execute("""
    INSERT INTO tickets
    (title, description, priority, status, assigned_to, created_by, created_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        title,
        description,
        priority,
        "Open",
        assigned_to,
        created_by,
        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ))

    db.commit()
    print("✅ Ticket successfully created")

# -------------------------------
# UPDATE STATUS WITH RULE
# -------------------------------
def update_ticket_status(ticket_id, new_status):
    cur.execute("SELECT status FROM tickets WHERE ticket_id=?", (ticket_id,))
    current = cur.fetchone()

    if not current:
        print("❌ Ticket not found")
        return

    if current[0] == "Open" and new_status == "Done":
        print("🚫 Invalid transition: Open → Done not allowed")
        return

    cur.execute("""
    UPDATE tickets SET status=? WHERE ticket_id=?
    """, (new_status, ticket_id))

    db.commit()
    print("✅ Status updated")

# -------------------------------
# DISPLAY TICKETS
# -------------------------------
def show_tickets():
    cur.execute("""
    SELECT ticket_id, title, priority, status, created_time
    FROM tickets ORDER BY created_time DESC
    """)
    tickets = cur.fetchall()
    if tickets:
        print("\n--- Current Support Tickets ---")
        for ticket in tickets:
            print(f"ID: {ticket[0]}, Title: {ticket[1]}, Priority: {ticket[2]}, Status: {ticket[3]}, Created: {ticket[4]}")
        print("-------------------------------")
    else:
        print("No tickets found.")

In [ ]:
import csv

def load_tickets_from_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            create_ticket(
                title=row["title"],
                description=row["description"],
                priority=row["priority"],
                assigned_to=row["assigned_to"],
                created_by=row["created_by"]
            )



In [ ]:
load_tickets_from_csv("/sample_support_tickets.csv")
show_tickets()


⚠️ Similar issue detected. Please verify before proceeding.
✅ Ticket successfully created
⚠️ Similar issue detected. Please verify before proceeding.
✅ Ticket successfully created
⚠️ Similar issue detected. Please verify before proceeding.
✅ Ticket successfully created
⚠️ Similar issue detected. Please verify before proceeding.
✅ Ticket successfully created
⚠️ Similar issue detected. Please verify before proceeding.
✅ Ticket successfully created

--- Current Support Tickets ---
ID: 22, Title: Login failure at night, Priority: Medium, Status: Open, Created: 2026-01-14 05:36:32
ID: 23, Title: Database connection error, Priority: High, Status: Open, Created: 2026-01-14 05:36:32
ID: 24, Title: Slow application response, Priority: Low, Status: Open, Created: 2026-01-14 05:36:32
ID: 25, Title: Password reset issue, Priority: Medium, Status: Open, Created: 2026-01-14 05:36:32
ID: 21, Title: Server login issue, Priority: High, Status: Open, Created: 2026-01-14 05:36:31
ID: 16, Title: Server lo

To install a Python library, you can use the `pip install` command directly in a code cell by preceding it with an exclamation mark `!`. This tells Colab to run the command in the shell.

In [ ]:
# Example: Install the pandas library
!pip install pandas

In [ ]:
from google.colab import files
files.download("support_tickets.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cur.execute("SELECT * FROM tickets")
rows = cur.fetchall()

print("Total records saved:", len(rows))
for row in rows:
    print(row)


Total records saved: 25
(1, 'Server login issue', 'Users unable to login during night shift', 'High', 'Open', 'support@company.com', 'admin@company.com', '2026-01-14 05:32:15')
(2, 'Login failure at night', 'Authentication errors after midnight', 'Medium', 'Open', 'support@company.com', 'admin@company.com', '2026-01-14 05:32:15')
(3, 'Database connection error', 'Application cannot connect to database server', 'High', 'Open', 'dbteam@company.com', 'ops@company.com', '2026-01-14 05:32:15')
(4, 'Slow application response', 'Application performance degrades during peak hours', 'Low', 'Open', 'support@company.com', 'ops@company.com', '2026-01-14 05:32:15')
(5, 'Password reset issue', 'Users unable to reset password through portal', 'Medium', 'Open', 'support@company.com', 'admin@company.com', '2026-01-14 05:32:15')
(6, 'Server login issue', 'Users unable to login during night shift', 'High', 'Open', 'support@company.com', 'admin@company.com', '2026-01-14 05:34:25')
(7, 'Login failure at ni

In [ ]:
def create_ticket(title, description, priority, assigned_to, created_by):
    combined = title + " " + description

    if is_similar_issue(combined):
        print("⚠️ Similar issue detected. Please verify before proceeding.")

    cur.execute("""
    INSERT INTO tickets
    (title, description, priority, status, assigned_to, created_by, created_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        title,
        description,
        priority,
        "Open",
        assigned_to,
        created_by,
        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ))

    db.commit()

    # ✅ Confirmation Log
    print("✅ DATA SAVED TO DATABASE")
    print(f"   Title       : {title}")
    print(f"   Priority    : {priority}")
    print(f"   Assigned To : {assigned_to}")


In [ ]:
def verify_database():
    cur.execute("SELECT COUNT(*) FROM tickets")
    count = cur.fetchone()[0]
    print(f"📦 Total records in database: {count}")

verify_database()


📦 Total records in database: 25


After running the cell, the library will be installed and ready to be imported and used in subsequent cells.